<a href="https://colab.research.google.com/github/jwlai-cloud/DataScience_Playground/blob/master/IP_range_to_CIDR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import ipaddress
from google.colab import data_table


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

Authenticated


In [ ]:
%%bigquery --project tgds-dev
#whitelist IP
CREATE
OR REPLACE TABLE `tgds-dev.dev_junwei.whitelist_ip` AS WITH isp AS(
    SELECT
        isp_organization,
        max(invalid_percentage) invalid_percentage,
        MAX(known_invalid_percentage) known_invalid_percentage,
        sum(COUNT_ppc_invalid_click_known_reason) COUNT_ppc_invalid_click_known_reason,
        sum(COUNT_ppc_successful_click) COUNT_ppc_successful_click,
        SUM(COUNT_ppc_invalid_click) COUNT_ppc_invalid_click
    FROM (
            SELECT
                -- bottlenose_hash_key ip_address,
                isp_organization,
                SUM(is_wifi) is_wifi,
                SUM(is_anonymous) is_anonymous,
                SUM(is_anonymous_vpn) is_anonymous_vpn,
                SUM(is_hosting_provider) is_hosting_provider,
                SUM(is_public_proxy) is_public_proxy,
                SUM(is_tor_exit_node) is_tor_exit_node,
                SUM(COUNT_click) COUNT_click,
                SUM(COUNT_ppc_invalid_click) COUNT_ppc_invalid_click,
                round(
                    safe_divide(
                        SUM(COUNT_ppc_invalid_click) * 100.00,
                        SUM(COUNT_ppc_successful_click)
                    ),
                    2
                ) invalid_percentage,
                SUM(COUNT_ppc_successful_click) COUNT_ppc_successful_click,
                SUM(COUNT_ppc_invalid_click_known_reason) COUNT_ppc_invalid_click_known_reason,
                round(
                    safe_divide(
                        SUM(COUNT_ppc_invalid_click_known_reason) * 100.00,
                        SUM(COUNT_ppc_successful_click)
                    ),
                    2
                ) known_invalid_percentage
            FROM `tg-stage.tg_bottlenose_profile_ds.tg_bottlenose_ip_address_ppc`
            WHERE date(bottlenose_partition_day) = "2021-08-18"
              AND lower(country_name) LIKE 'united%kingdom'
            GROUP BY 1
          ORDER BY COUNT_click DESC
        )
    WHERE known_invalid_percentage <= 0.01
      AND invalid_percentage <= 15
    GROUP BY 1
    HAVING     COUNT_ppc_successful_click >= 500
  ORDER BY COUNT_ppc_successful_click DESC
),
whitelist_ip AS(
    SELECT
        NET.IP_TO_STRING(
            NET.IP_TRUNC(NET.SAFE_IP_FROM_STRING(bottlenose_hash_key), 24)
        ) ip_range,
        isp_organization
    FROM `tg-stage.tg_bottlenose_profile_ds.tg_bottlenose_ip_address_ppc`
    WHERE date(bottlenose_partition_day) = "2021-08-18"
      AND lower(country_name) LIKE 'united%kingdom'
      AND bottlenose_hash_key NOT LIKE '%:%'
      AND isp_organization IN (
            SELECT
                isp_organization
            FROM isp
        )
    GROUP BY 1,
        2
)
SELECT
    NET.IPV4_TO_INT64(NET.IP_FROM_STRING(ip_range)) ip_range_int,
    ip_range,
    string_agg(isp_organization, '\n') AS isp_organization
FROM whitelist_ip
GROUP BY ip_range
ORDER BY ip_range_int;

SELECT @@error.message;

,f0_
0,None


In [ ]:
# Save output in a variable 

%%bigquery --project tgds-dev smart_black_df
select *
from `tgds-dev.dev_junwei.smart_black_ip_range_info`

In [ ]:
smart_black_df = smart_black_df[['SmartIP_first', 
         'SmartIP_last', 
         'ExtIP_first',
         'ExtIP_last'
]]

print(smart_black_df.shape)

(172, 4)


In [ ]:
%load_ext google.colab.data_table
smart_black_df

,SmartIP_first,SmartIP_last,ExtIP_first,ExtIP_last
0,2.56.148.0,2.58.47.255,2.56.0.0,2.58.255.255
1,5.61.60.0,5.61.63.255,5.61.0.0,5.61.255.255
2,5.157.132.0,5.157.255.255,5.157.0.0,5.157.255.255
3,5.180.76.0,5.183.179.255,5.180.0.0,5.183.255.255
4,5.252.196.0,5.253.207.255,5.252.0.0,5.253.255.255
...,...,...,...,...
167,216.177.133.0,216.189.124.255,216.177.0.0,216.189.255.255
168,217.9.16.0,217.9.19.255,217.9.0.0,217.9.255.255
169,217.64.112.0,217.64.127.255,217.64.0.0,217.64.255.255
170,217.138.192.0,217.151.111.255,217.138.0.0,217.151.255.255


In [ ]:
len(smart_black_df)

172

In [ ]:
# Save output in a variable 

%%bigquery --project tgds-dev raw_black_df
select *
from `tgds-dev.dev_junwei.raw_black_ip_range_info`

In [ ]:
display(raw_black_df)

,ip_range,autonomous_system_number,autonomous_system_organization,isp,organization,country_name,cidr,IPint_from_str,IPint_first,IPint_last,IPstr_first,IPstr_last,IPint_first_prev,IPint_last_prev,IP_gap,threshold,RawIP_minmax,group_id
0,2.56.148.0/22,9009,M247 Ltd,M247 Ltd,M247 Ltd,None,22,37262336,37262336,37263359,2.56.148.0,2.56.151.255,NaN,NaN,NaN,0,2.56.148.0-2.56.151.255,0
1,2.58.44.0/22,9009,M247 Ltd,M247 Ltd,M247 Ltd,None,22,37366784,37366784,37367807,2.58.44.0,2.58.47.255,3.726234e+07,3.726336e+07,103425.0,0,2.58.44.0-2.58.47.255,0
2,5.61.60.0/22,9009,M247 Ltd,M247 Ltd,M247 Ltd,United Kingdom,22,87899136,87899136,87900159,5.61.60.0,5.61.63.255,3.736678e+07,3.736781e+07,50531329.0,1,5.61.60.0-5.61.63.255,1
3,5.157.132.0/22,9009,M247 Ltd,M247 Ltd,M247 Ltd,None,22,94209024,94209024,94210047,5.157.132.0,5.157.135.255,8.789914e+07,8.790016e+07,6308865.0,1,5.157.132.0-5.157.135.255,2
4,5.157.152.0/21,9009,M247 Ltd,M247 Ltd,M247 Ltd,None,21,94214144,94214144,94216191,5.157.152.0,5.157.159.255,9.420902e+07,9.421005e+07,4097.0,0,5.157.152.0-5.157.159.255,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
788,217.138.220.0/23,9009,M247 Ltd,M247 Ltd,M247 Ltd,None,23,3649756160,3649756160,3649756671,217.138.220.0,217.138.221.255,3.649755e+09,3.649756e+09,1.0,0,217.138.220.0-217.138.221.255,170
789,217.138.252.0/22,9009,M247 Ltd,M247 Ltd,M247 Ltd,None,22,3649764352,3649764352,3649765375,217.138.252.0,217.138.255.255,3.649756e+09,3.649757e+09,7681.0,0,217.138.252.0-217.138.255.255,170
790,217.148.140.0/22,9009,M247 Ltd,M247 Ltd,M247 Ltd,None,22,3650391040,3650391040,3650392063,217.148.140.0,217.148.143.255,3.649764e+09,3.649765e+09,625665.0,0,217.148.140.0-217.148.143.255,170
791,217.151.96.0/20,9009,M247 Ltd,M247 Ltd,M247 Ltd,None,20,3650576384,3650576384,3650580479,217.151.96.0,217.151.111.255,3.650391e+09,3.650392e+09,184321.0,0,217.151.96.0-217.151.111.255,170


In [ ]:
raw_black_network = pd.Series([ipaddress.IPv4Network(item) for item in raw_black_df['ip_range']])

In [ ]:
# Save output in a variable 

%%bigquery --project tgds-dev whitelist_df
select *
from `tgds-dev.dev_junwei.whitelist_ip`

In [ ]:
len(whitelist_df)

139657

In [ ]:
whitelist_df

,ip_range_int,ip_range,isp_organization
0,35127296,2.24.0.0,EE
1,35127552,2.24.1.0,EE
2,35127808,2.24.2.0,EE
3,35128064,2.24.3.0,EE
4,35128320,2.24.4.0,EE
...,...,...,...
139652,3652525056,217.181.28.0,RM Education Ltd
139653,3652525568,217.181.30.0,RM Education Ltd
139654,3652530176,217.181.48.0,RM Education Ltd
139655,3652532736,217.181.58.0,RM Education Ltd


In [ ]:
white_ipint = whitelist_df['ip_range_int'].values

In [ ]:
# Save output in a variable 

%%bigquery --project tgds-dev IP_Metric_TG_df
select *
from `tgds-dev.dev_junwei.ip_address_metric_wh`

In [ ]:
len(IP_Metric_TG_df)

157331

In [ ]:
IP_Metric_TG_df.head()

,root_element_user_ip_info_autonomous_system_organization,ip_range,ip_range_int,total_click,success_click,invalid_click,count_ppc_invalid_click_known_reason,invalid_percentage,known_invalid_percentage,watchlist_count,total_conversion_event_journey
0,EE Limited,213.205.192.0,3.587031e+09,47239,41179,6060,6,12.83,0.01,1457,14436.0
1,EE Limited,213.205.241.0,3.587044e+09,40311,35594,4717,2,11.70,0.00,1003,9476.0
2,EE Limited,213.205.242.0,3.587044e+09,40086,35068,5018,0,12.52,0.00,1099,12537.0
3,EE Limited,213.205.198.0,3.587033e+09,35595,31427,4168,4,11.71,0.01,613,7768.0
4,EE Limited,213.205.194.0,3.587032e+09,35491,31331,4160,3,11.72,0.01,754,7263.0


In [ ]:
ip_range=[]

In [ ]:
# for row in df.itertuples():
#   ip_range.append([str(ipaddr)  for ipaddr in 
#                           ipaddress.summarize_address_range(ipaddress.IPv4Address(row.SmartIP_first), 
#                                                             ipaddress.IPv4Address(row.SmartIP_last)) 
#                           ])
  
for row in smart_black_df.itertuples():
  ip_range.append([(ipaddr, row.ExtIP_first,row.ExtIP_last)   for ipaddr in 
                          ipaddress.summarize_address_range(ipaddress.IPv4Address(row.ExtIP_first), 
                                                            ipaddress.IPv4Address(row.ExtIP_last)) 
                          ])

In [ ]:
ip_range[0]

[(IPv4Network('2.56.0.0/15'), '2.56.0.0', '2.58.255.255'),
 (IPv4Network('2.58.0.0/16'), '2.56.0.0', '2.58.255.255')]

In [ ]:
ip_range_flat = [(ipaddr,compr_first,compr_last ) for item in ip_range for (ipaddr, compr_first, compr_last ) in item ]

In [ ]:
print(f"The total number of IP CIDR ranges after compression is {len(ip_range_flat)}")

The total number of IP CIDR ranges after compression is 300


In [ ]:
ip_range_flat_str = [str(ipr[0]) for ipr in ip_range_flat]
ip_range_flat_min_ipstr = [str(ipr[0][0]) for ipr in ip_range_flat]
ip_range_flat_max_ipstr = [str(ipr[0][-1]) for ipr in ip_range_flat]
ip_range_flat_min_ipint = [int(ipr[0][0]) for ipr in ip_range_flat]
ip_range_flat_max_ipint = [int(ipr[0][-1]) for ipr in ip_range_flat]
ip_range_flat_min_compripstr = [ipr[1] for ipr in ip_range_flat]
ip_range_flat_max_compripstr = [ipr[2] for ipr in ip_range_flat]
ip_range_flat_int_pair = [(int(ipr[0][0]), int(ipr[0][-1])) for ipr in ip_range_flat]

In [ ]:
cidr_ip_range_df = pd.DataFrame({
    "IP_range_str": ip_range_flat_str,
    "ExtIP_first": ip_range_flat_min_ipstr,
    "ExtIP_last": ip_range_flat_max_ipstr,
    "ExtIP_int_first": ip_range_flat_min_ipint,
    "ExtIP_int_last": ip_range_flat_max_ipint,
    "ComprIP_first": ip_range_flat_min_compripstr,
    "ComprIP_last": ip_range_flat_max_compripstr,
    "IP_Pair": ip_range_flat_int_pair
})

In [ ]:
cidr_ip_range_df

,IP_range_str,ExtIP_first,ExtIP_last,ExtIP_int_first,ExtIP_int_last,ComprIP_first,ComprIP_last,IP_Pair
0,2.56.0.0/15,2.56.0.0,2.57.255.255,37224448,37355519,2.56.0.0,2.58.255.255,"(37224448, 37355519)"
1,2.58.0.0/16,2.58.0.0,2.58.255.255,37355520,37421055,2.56.0.0,2.58.255.255,"(37355520, 37421055)"
2,5.61.0.0/16,5.61.0.0,5.61.255.255,87883776,87949311,5.61.0.0,5.61.255.255,"(87883776, 87949311)"
3,5.157.0.0/16,5.157.0.0,5.157.255.255,94175232,94240767,5.157.0.0,5.157.255.255,"(94175232, 94240767)"
4,5.180.0.0/14,5.180.0.0,5.183.255.255,95682560,95944703,5.180.0.0,5.183.255.255,"(95682560, 95944703)"
...,...,...,...,...,...,...,...,...
295,217.64.0.0/16,217.64.0.0,217.64.255.255,3644850176,3644915711,217.64.0.0,217.64.255.255,"(3644850176, 3644915711)"
296,217.138.0.0/15,217.138.0.0,217.139.255.255,3649699840,3649830911,217.138.0.0,217.151.255.255,"(3649699840, 3649830911)"
297,217.140.0.0/14,217.140.0.0,217.143.255.255,3649830912,3650093055,217.138.0.0,217.151.255.255,"(3649830912, 3650093055)"
298,217.144.0.0/13,217.144.0.0,217.151.255.255,3650093056,3650617343,217.138.0.0,217.151.255.255,"(3650093056, 3650617343)"


In [ ]:
num_whiteIPrange = len(whitelist_df)

In [ ]:
def is_white_inside(col):
  hits_sum = whitelist_df[(col[0]<=whitelist_df['ip_range_int']) & (whitelist_df['ip_range_int']<=col[1])]['ip_range_int'].count()
  hits_rate = hits_sum / num_whiteIPrange
  return (hits_sum, hits_rate)

In [ ]:
def metric_inside(col):
  metrics = tuple(IP_Metric_TG_df[(col[0]<=IP_Metric_TG_df['ip_range_int']) & (IP_Metric_TG_df['ip_range_int']<=col[1])]
                  [['ip_range',
                    'total_click', 
                    'success_click',
                    'invalid_click',
                    'count_ppc_invalid_click_known_reason',
                    'watchlist_count',
                    'total_conversion_event_journey']]
                  .agg({'ip_range': 'count', 
                        'total_click': 'sum', 
                        'success_click': 'sum',
                        'invalid_click': 'sum',
                        'count_ppc_invalid_click_known_reason': 'sum',
                        'watchlist_count': 'sum',
                        'total_conversion_event_journey': 'sum'}))
  return metrics

In [ ]:
def is_raw_black_inside(col):
  raw_hit_df = raw_black_network[raw_black_network.apply(lambda x: x.overlaps(col))]
  hits_sum = raw_hit_df.count()
  hits_numIP = raw_hit_df.apply(lambda x: x.num_addresses).sum()
  return (hits_sum, hits_numIP)

In [ ]:
cidr_ip_range_df['network']= cidr_ip_range_df['IP_range_str'].apply(ipaddress.IPv4Network)

In [ ]:
cidr_ip_range_df['Raw_black_ip_qc'] = cidr_ip_range_df['network'].apply(is_raw_black_inside)

In [ ]:
cidr_ip_range_df

,IP_range_str,ExtIP_first,ExtIP_last,ExtIP_int_first,ExtIP_int_last,ComprIP_first,ComprIP_last,IP_Pair,network,Raw_black_ip_qc
0,2.56.0.0/15,2.56.0.0,2.57.255.255,37224448,37355519,2.56.0.0,2.58.255.255,"(37224448, 37355519)",2.56.0.0/15,"(1, 1024)"
1,2.58.0.0/16,2.58.0.0,2.58.255.255,37355520,37421055,2.56.0.0,2.58.255.255,"(37355520, 37421055)",2.58.0.0/16,"(1, 1024)"
2,5.61.0.0/16,5.61.0.0,5.61.255.255,87883776,87949311,5.61.0.0,5.61.255.255,"(87883776, 87949311)",5.61.0.0/16,"(1, 1024)"
3,5.157.0.0/16,5.157.0.0,5.157.255.255,94175232,94240767,5.157.0.0,5.157.255.255,"(94175232, 94240767)",5.157.0.0/16,"(7, 14336)"
4,5.180.0.0/14,5.180.0.0,5.183.255.255,95682560,95944703,5.180.0.0,5.183.255.255,"(95682560, 95944703)",5.180.0.0/14,"(7, 6656)"
...,...,...,...,...,...,...,...,...,...,...
295,217.64.0.0/16,217.64.0.0,217.64.255.255,3644850176,3644915711,217.64.0.0,217.64.255.255,"(3644850176, 3644915711)",217.64.0.0/16,"(1, 4096)"
296,217.138.0.0/15,217.138.0.0,217.139.255.255,3649699840,3649830911,217.138.0.0,217.151.255.255,"(3649699840, 3649830911)",217.138.0.0/15,"(6, 8192)"
297,217.140.0.0/14,217.140.0.0,217.143.255.255,3649830912,3650093055,217.138.0.0,217.151.255.255,"(3649830912, 3650093055)",217.140.0.0/14,"(0, 0)"
298,217.144.0.0/13,217.144.0.0,217.151.255.255,3650093056,3650617343,217.138.0.0,217.151.255.255,"(3650093056, 3650617343)",217.144.0.0/13,"(2, 5120)"


In [ ]:
cidr_ip_range_df[['Raw_BlackIPrange_hits', 'Raw_BlackIP_num']] = pd.DataFrame([item for item in cidr_ip_range_df['Raw_black_ip_qc']])

In [ ]:
cidr_ip_range_df.sort_values(by='Raw_BlackIPrange_hits', ascending=False)

,IP_range_str,ExtIP_first,ExtIP_last,ExtIP_int_first,ExtIP_int_last,ComprIP_first,ComprIP_last,IP_Pair,network,Raw_black_ip_qc,Raw_BlackIPrange_hits,Raw_BlackIP_num
216,185.128.0.0/9,185.128.0.0,185.255.255.255,3112173568,3120562175,185.75.0.0,185.255.255.255,"(3112173568, 3120562175)",185.128.0.0/9,"(93, 58624)",93,58624
33,45.128.0.0/12,45.128.0.0,45.143.255.255,763363328,764411903,45.128.0.0,45.157.255.255,"(763363328, 764411903)",45.128.0.0/12,"(25, 14592)",25,14592
29,45.80.0.0/13,45.80.0.0,45.87.255.255,760217600,760741887,45.59.0.0,45.94.255.255,"(760217600, 760741887)",45.80.0.0/13,"(24, 20480)",24,20480
187,172.96.0.0/12,172.96.0.0,172.111.255.255,2891972608,2893021183,172.84.0.0,172.111.255.255,"(2891972608, 2893021183)",172.96.0.0/12,"(23, 9728)",23,9728
215,185.96.0.0/11,185.96.0.0,185.127.255.255,3110076416,3112173567,185.75.0.0,185.255.255.255,"(3110076416, 3112173567)",185.96.0.0/11,"(22, 14080)",22,14080
...,...,...,...,...,...,...,...,...,...,...,...,...
95,89.240.0.0/13,89.240.0.0,89.247.255.255,1508900864,1509425151,89.238.0.0,89.249.255.255,"(1508900864, 1509425151)",89.240.0.0/13,"(0, 0)",0,0
60,79.112.0.0/13,79.112.0.0,79.119.255.255,1332740096,1333264383,79.110.0.0,79.124.255.255,"(1332740096, 1333264383)",79.112.0.0/13,"(0, 0)",0,0
54,77.88.0.0/15,77.88.0.0,77.89.255.255,1297612800,1297743871,77.81.0.0,77.90.255.255,"(1297612800, 1297743871)",77.88.0.0/15,"(0, 0)",0,0
53,77.84.0.0/14,77.84.0.0,77.87.255.255,1297350656,1297612799,77.81.0.0,77.90.255.255,"(1297350656, 1297612799)",77.84.0.0/14,"(0, 0)",0,0


In [ ]:
cidr_ip_range_df['WhiteIP_hits'] = cidr_ip_range_df.IP_Pair.apply(is_white_inside)

In [ ]:
cidr_ip_range_df[['WhiteIP_hits_total', 'WhiteIP_hits_rate']] = pd.DataFrame([item for item in cidr_ip_range_df['WhiteIP_hits']])

In [ ]:
cidr_ip_range_df['WhiteIP_num'] = cidr_ip_range_df['WhiteIP_hits_total'].mul(256*256)

In [ ]:
cidr_ip_range_df['Black/White IP ratio'] = cidr_ip_range_df['Raw_BlackIP_num'].div(cidr_ip_range_df['WhiteIP_num'], axis='index')

In [ ]:
cidr_ip_range_df.query('Raw_BlackIPrange_hits > 0', inplace=True)

In [ ]:
# need to reset index, otherwise future operation using "pd.DataFrame([item for item in DataFrame])" will be wrong since that 
# will use reset index
cidr_ip_range_df.reset_index(inplace=True)

In [ ]:
%reload_ext google.colab.data_table
cidr_ip_range_df

,index,IP_range_str,ExtIP_first,ExtIP_last,ExtIP_int_first,ExtIP_int_last,ComprIP_first,ComprIP_last,IP_Pair,network,Raw_black_ip_qc,Raw_BlackIPrange_hits,Raw_BlackIP_num,WhiteIP_hits,WhiteIP_hits_total,WhiteIP_hits_rate,WhiteIP_num,Black/White IP ratio
0,0,2.56.0.0/15,2.56.0.0,2.57.255.255,37224448,37355519,2.56.0.0,2.58.255.255,"(37224448, 37355519)",2.56.0.0/15,"(1, 1024)",1,1024,"(0, 0.0)",0,0.000000,0,inf
1,1,2.58.0.0/16,2.58.0.0,2.58.255.255,37355520,37421055,2.56.0.0,2.58.255.255,"(37355520, 37421055)",2.58.0.0/16,"(1, 1024)",1,1024,"(0, 0.0)",0,0.000000,0,inf
2,2,5.61.0.0/16,5.61.0.0,5.61.255.255,87883776,87949311,5.61.0.0,5.61.255.255,"(87883776, 87949311)",5.61.0.0/16,"(1, 1024)",1,1024,"(3, 2.1481200369476646e-05)",3,0.000021,196608,0.005208
3,3,5.157.0.0/16,5.157.0.0,5.157.255.255,94175232,94240767,5.157.0.0,5.157.255.255,"(94175232, 94240767)",5.157.0.0/16,"(7, 14336)",7,14336,"(0, 0.0)",0,0.000000,0,inf
4,4,5.180.0.0/14,5.180.0.0,5.183.255.255,95682560,95944703,5.180.0.0,5.183.255.255,"(95682560, 95944703)",5.180.0.0/14,"(7, 6656)",7,6656,"(6, 4.296240073895329e-05)",6,0.000043,393216,0.016927
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
261,294,217.9.0.0/16,217.9.0.0,217.9.255.255,3641245696,3641311231,217.9.0.0,217.9.255.255,"(3641245696, 3641311231)",217.9.0.0/16,"(1, 1024)",1,1024,"(0, 0.0)",0,0.000000,0,inf
262,295,217.64.0.0/16,217.64.0.0,217.64.255.255,3644850176,3644915711,217.64.0.0,217.64.255.255,"(3644850176, 3644915711)",217.64.0.0/16,"(1, 4096)",1,4096,"(0, 0.0)",0,0.000000,0,inf
263,296,217.138.0.0/15,217.138.0.0,217.139.255.255,3649699840,3649830911,217.138.0.0,217.151.255.255,"(3649699840, 3649830911)",217.138.0.0/15,"(6, 8192)",6,8192,"(155, 0.0011098620190896267)",155,0.001110,10158080,0.000806
264,298,217.144.0.0/13,217.144.0.0,217.151.255.255,3650093056,3650617343,217.138.0.0,217.151.255.255,"(3650093056, 3650617343)",217.144.0.0/13,"(2, 5120)",2,5120,"(30, 0.00021481200369476646)",30,0.000215,1966080,0.002604


In [ ]:
cidr_ip_range_df['IP_Metric_QC'] = cidr_ip_range_df.IP_Pair.apply(metric_inside)

In [ ]:
cidr_ip_range_df[['Metric_hits_total', 
                  'Total_click', 
                  'Success_click',
                  'Invalid_click',
                  'Count_ppc_invalid_click_known_reason',
                  'Watchlist_count',
                  'Total_conversion_event_journey']] = pd.DataFrame([item for item in cidr_ip_range_df['IP_Metric_QC']])

In [ ]:
cidr_ip_range_df = cidr_ip_range_df.drop(columns=['network', 'IP_Pair', 'Raw_black_ip_qc', 'WhiteIP_hits','IP_Metric_QC'])

In [ ]:
cidr_ip_range_df.sort_values(by='WhiteIP_hits_total', ascending=False).head()

,index,IP_range_str,ExtIP_first,ExtIP_last,ExtIP_int_first,ExtIP_int_last,ComprIP_first,ComprIP_last,Raw_BlackIPrange_hits,Raw_BlackIP_num,WhiteIP_hits_total,WhiteIP_hits_rate,WhiteIP_num,Black/White IP ratio,Metric_hits_total,Total_click,Success_click,Invalid_click,Count_ppc_invalid_click_known_reason,Watchlist_count,Total_conversion_event_journey
226,252,194.64.0.0/10,194.64.0.0,194.127.255.255,3258974208,3263168511,194.31.0.0,194.127.255.255,14,8448,689,0.004934,45154304,0.000187,787.0,14630.0,7094.0,7536.0,6939.0,6125.0,459.0
263,296,217.138.0.0/15,217.138.0.0,217.139.255.255,3649699840,3649830911,217.138.0.0,217.151.255.255,6,8192,155,0.001110,10158080,0.000806,164.0,1380.0,800.0,580.0,522.0,50.0,583.0
194,216,185.128.0.0/9,185.128.0.0,185.255.255.255,3112173568,3120562175,185.75.0.0,185.255.255.255,93,58624,138,0.000988,9043968,0.006482,622.0,25563.0,15042.0,10521.0,7002.0,4380.0,2175.0
193,215,185.96.0.0/11,185.96.0.0,185.127.255.255,3110076416,3112173567,185.75.0.0,185.255.255.255,22,14080,56,0.000401,3670016,0.003836,242.0,10194.0,3601.0,6593.0,6322.0,5404.0,272.0
186,207,185.16.0.0/12,185.16.0.0,185.31.255.255,3104833536,3105882111,185.4.0.0,185.42.255.255,3,2560,48,0.000344,3145728,0.000814,128.0,2946.0,2422.0,524.0,308.0,35.0,457.0


In [ ]:
data_table.DataTable(cidr_ip_range_df)

,index,IP_range_str,ExtIP_first,ExtIP_last,ExtIP_int_first,ExtIP_int_last,ComprIP_first,ComprIP_last,Raw_BlackIPrange_hits,Raw_BlackIP_num,WhiteIP_hits_total,WhiteIP_hits_rate,WhiteIP_num,Black/White IP ratio,Metric_hits_total,Total_click,Success_click,Invalid_click,Count_ppc_invalid_click_known_reason,Watchlist_count,Total_conversion_event_journey
0,0,2.56.0.0/15,2.56.0.0,2.57.255.255,37224448,37355519,2.56.0.0,2.58.255.255,1,1024,0,0.000000,0,inf,7.0,159.0,135.0,24.0,3.0,2.0,0.0
1,1,2.58.0.0/16,2.58.0.0,2.58.255.255,37355520,37421055,2.56.0.0,2.58.255.255,1,1024,0,0.000000,0,inf,6.0,189.0,14.0,175.0,172.0,16.0,0.0
2,2,5.61.0.0/16,5.61.0.0,5.61.255.255,87883776,87949311,5.61.0.0,5.61.255.255,1,1024,3,0.000021,196608,0.005208,10.0,347.0,329.0,18.0,0.0,0.0,0.0
3,3,5.157.0.0/16,5.157.0.0,5.157.255.255,94175232,94240767,5.157.0.0,5.157.255.255,7,14336,0,0.000000,0,inf,7.0,73.0,65.0,8.0,0.0,0.0,3.0
4,4,5.180.0.0/14,5.180.0.0,5.183.255.255,95682560,95944703,5.180.0.0,5.183.255.255,7,6656,6,0.000043,393216,0.016927,20.0,462.0,219.0,243.0,216.0,83.0,17.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
261,294,217.9.0.0/16,217.9.0.0,217.9.255.255,3641245696,3641311231,217.9.0.0,217.9.255.255,1,1024,0,0.000000,0,inf,17.0,5980.0,85.0,5895.0,5892.0,5559.0,11.0
262,295,217.64.0.0/16,217.64.0.0,217.64.255.255,3644850176,3644915711,217.64.0.0,217.64.255.255,1,4096,0,0.000000,0,inf,5.0,19.0,13.0,6.0,5.0,0.0,0.0
263,296,217.138.0.0/15,217.138.0.0,217.139.255.255,3649699840,3649830911,217.138.0.0,217.151.255.255,6,8192,155,0.001110,10158080,0.000806,164.0,1380.0,800.0,580.0,522.0,50.0,583.0
264,298,217.144.0.0/13,217.144.0.0,217.151.255.255,3650093056,3650617343,217.138.0.0,217.151.255.255,2,5120,30,0.000215,1966080,0.002604,65.0,2034.0,456.0,1578.0,1513.0,327.0,31.0


In [ ]:
cidr_ip_range_df.describe()

,index,ExtIP_int_first,ExtIP_int_last,Raw_BlackIPrange_hits,Raw_BlackIP_num,WhiteIP_hits_total,WhiteIP_hits_rate,WhiteIP_num,Black/White IP ratio,Metric_hits_total,Total_click,Success_click,Invalid_click,Count_ppc_invalid_click_known_reason,Watchlist_count,Total_conversion_event_journey
count,266.000000,2.660000e+02,2.660000e+02,266.000000,266.000000,266.000000,266.000000,2.660000e+02,266.000000,266.000000,266.000000,266.000000,266.000000,266.000000,266.000000,266.000000
mean,146.917293,2.144487e+09,2.144733e+09,2.981203,2397.353383,6.503759,0.000047,4.262304e+05,inf,15.086466,604.541353,236.078947,368.462406,338.654135,255.721805,35.191729
std,87.317252,1.026562e+09,1.026632e+09,6.642035,4750.165935,44.614428,0.000319,2.923851e+06,NaN,65.810735,2282.329950,1169.763339,1414.364937,1313.341261,1162.026612,161.137387
min,0.000000,3.722445e+07,3.735552e+07,1.000000,256.000000,0.000000,0.000000,0.000000e+00,0.000187,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,71.250000,1.399620e+09,1.399685e+09,1.000000,512.000000,0.000000,0.000000,0.000000e+00,0.078125,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,148.500000,1.828028e+09,1.828094e+09,1.000000,1024.000000,0.000000,0.000000,0.000000e+00,inf,2.000000,11.000000,0.500000,1.500000,0.000000,0.000000,0.000000
75%,220.750000,3.169042e+09,3.169157e+09,3.000000,2048.000000,1.000000,0.000007,6.553600e+04,inf,7.000000,180.500000,64.000000,37.750000,19.500000,4.000000,2.750000
max,299.000000,3.653566e+09,3.653632e+09,93.000000,58624.000000,689.000000,0.004934,4.515430e+07,inf,787.000000,25563.000000,15042.000000,11624.000000,11624.000000,10894.000000,2175.000000


In [ ]:
print(f"Final false positive rate is {cidr_ip_range_df.WhiteIP_hits_total.sum()/num_whiteIPrange:.3%}")

Final false positive rate is 1.239%


In [ ]:
cidr_ip_range_df.to_csv('cidr_iprange_qc.csv')

In [ ]:
len(cidr_ip_range_df.query('WhiteIP_hits_total <= 0')['IP_range_str'])

198

In [ ]:
cidr_ip_range_df.query('WhiteIP_hits_total <= 0')['IP_range_str'].to_csv('cidr_iprange_no_false_positive.csv')

In [ ]:
data_table.DataTable(cidr_ip_range_df.query('WhiteIP_hits_total <= 0').sort_values(by='Raw_BlackIPrange_hits', ascending=False))

,index,IP_range_str,ExtIP_first,ExtIP_last,ExtIP_int_first,ExtIP_int_last,ComprIP_first,ComprIP_last,Raw_BlackIPrange_hits,Raw_BlackIP_num,WhiteIP_hits_total,WhiteIP_hits_rate,WhiteIP_num,Black/White IP ratio,Metric_hits_total,Total_click,Success_click,Invalid_click,Count_ppc_invalid_click_known_reason,Watchlist_count,Total_conversion_event_journey
169,187,172.96.0.0/12,172.96.0.0,172.111.255.255,2891972608,2893021183,172.84.0.0,172.111.255.255,23,9728,0,0.0,0,inf,11.0,1084.0,2.0,1082.0,1082.0,490.0,0.0
105,116,93.120.0.0/16,93.120.0.0,93.120.255.255,1568145408,1568210943,93.115.0.0,93.120.255.255,18,13056,0,0.0,0,inf,0.0,0.0,0.0,0.0,0.0,0.0,0.0
19,19,38.200.0.0/14,38.200.0.0,38.203.255.255,650641408,650903551,38.200.0.0,38.206.255.255,12,12288,0,0.0,0,inf,0.0,0.0,0.0,0.0,0.0,0.0,0.0
14,14,37.120.0.0/16,37.120.0.0,37.120.255.255,628621312,628686847,37.120.0.0,37.120.255.255,11,17664,0,0.0,0,inf,6.0,634.0,39.0,595.0,572.0,30.0,58.0
50,50,77.36.0.0/16,77.36.0.0,77.36.255.255,1294204928,1294270463,77.36.0.0,77.36.255.255,10,15616,0,0.0,0,inf,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
104,115,93.116.0.0/14,93.116.0.0,93.119.255.255,1567883264,1568145407,93.115.0.0,93.120.255.255,1,256,0,0.0,0,inf,1.0,23.0,0.0,23.0,23.0,0.0,0.0
106,117,93.177.0.0/16,93.177.0.0,93.177.255.255,1571880960,1571946495,93.177.0.0,93.177.255.255,1,1024,0,0.0,0,inf,1.0,92.0,14.0,78.0,75.0,5.0,0.0
109,120,94.198.0.0/16,94.198.0.0,94.198.255.255,1590034432,1590099967,94.198.0.0,94.198.255.255,1,1024,0,0.0,0,inf,7.0,30.0,23.0,7.0,0.0,6.0,0.0
111,123,95.180.0.0/15,95.180.0.0,95.181.255.255,1605632000,1605763071,95.174.0.0,95.181.255.255,1,2048,0,0.0,0,inf,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
cidr_ip_range_final_list = list(cidr_ip_range_df[~(cidr_ip_range_df['WhiteIP_hits_total']>0)]['IP_range_str'])

In [ ]:
cidr_ip_range_final_str = '[' + ','.join(map(lambda x: "'"+x+"'",cidr_ip_range_final_list)) + ']'

In [ ]:
print(cidr_ip_range_final_str)

['2.56.0.0/15','2.58.0.0/16','5.157.0.0/16','24.235.0.0/16','31.13.0.0/16','31.14.0.0/16','31.133.0.0/16','31.134.0.0/15','31.204.0.0/16','37.35.0.0/16','37.97.0.0/16','37.120.0.0/16','37.221.0.0/16','38.65.0.0/16','38.95.0.0/16','38.132.0.0/16','38.200.0.0/14','38.204.0.0/15','38.206.0.0/16','45.9.0.0/16','45.10.0.0/15','45.12.0.0/14','45.33.0.0/16','45.59.0.0/16','45.60.0.0/14','45.92.0.0/15','45.156.0.0/15','45.250.0.0/16','50.114.0.0/16','52.144.0.0/16','62.182.0.0/16','62.216.0.0/16','63.246.0.0/16','64.43.0.0/16','64.79.0.0/16','66.171.0.0/16','69.51.0.0/16','69.168.0.0/16','77.36.0.0/16','77.81.0.0/16','77.82.0.0/15','77.90.0.0/16','77.232.0.0/13','78.31.0.0/16','79.110.0.0/15','79.124.0.0/16','80.83.0.0/16','81.16.0.0/16','81.90.0.0/15','81.161.0.0/16','82.102.0.0/15','82.112.0.0/14','83.143.0.0/16','83.171.0.0/16','84.22.0.0/16','84.39.0.0/16','84.247.0.0/16','85.8.0.0/16','85.120.0.0/15','85.203.0.0/16','85.204.0.0/16','86.62.0.0/16','86.106.0.0/16','87.101.0.0/16','88.218.0.

In [ ]:
from google.cloud import bigquery

client = bigquery.Client(project='tgds-dev')

result = client.query(f'''
CREATE
OR REPLACE TABLE `tgds-dev.dev_junwei.betfred_casino_smart_ip_range` AS 
SELECT
ip_range,
'betfred_casino' as organisation_identifier
from unnest({cidr_ip_range_final_str}) as ip_range
''')

result.to_dataframe()

""


In [ ]:
from google.cloud import bigquery

client = bigquery.Client(project='tgds-dev')

result = client.query(f'''
CREATE
OR REPLACE TABLE `tgds-dev.dev_junwei.betbull_sports_smart_ip_range` AS 
SELECT
ip_range,
'betbull_sports' as organisation_identifier
from unnest({cidr_ip_range_final_str}) as ip_range
''')

result.to_dataframe()

""


In [ ]:
from google.cloud import bigquery

client = bigquery.Client(project='tgds-dev')

result = client.query(f'''
SELECT
ip_range,
'betbull_sports' as organisation_identifier
from unnest({cidr_ip_range_final_list}) as ip_range
''')

result.to_dataframe()

,ip_range,organisation_identifier
0,2.56.0.0/15,betbull_sports
1,2.58.0.0/16,betbull_sports
2,5.157.0.0/16,betbull_sports
3,24.235.0.0/16,betbull_sports
4,31.13.0.0/16,betbull_sports
...,...,...
193,216.184.0.0/14,betbull_sports
194,216.188.0.0/15,betbull_sports
195,217.9.0.0/16,betbull_sports
196,217.64.0.0/16,betbull_sports
